In [11]:
from src.server.tibber import TibberClient
from gql import gql
import pandas as pd

In [7]:
client = TibberClient()

In [8]:
query = gql(
            """
            {
            viewer {
                homes {
                  currentSubscription{
                    priceInfo{
                      current{
                        total
                        energy
                        tax
                        startsAt
                      }
                      today {
                        total
                        energy
                        tax
                        startsAt
                      }
                      tomorrow {
                        total
                        energy
                        tax
                        startsAt
                      }
                    }
                  }
                }
              }
            }

        """
        )

result = await client.client.execute_async(query)

data = result["viewer"]["homes"][0]["currentSubscription"]["priceInfo"]

In [18]:
next_two_days = data["today"] + data["tomorrow"]
df = pd.DataFrame(next_two_days)
df["startsAt"] = pd.to_datetime(df["startsAt"])
df["hour"] = df["startsAt"].dt.hour
df["day"] = df["startsAt"].dt.day
df["Zeit"] = df["day"].astype(str) + ". " + df["hour"].astype(str)
df["Preis in ct"] = df["total"] * 100

In [21]:
df

,total,energy,tax,startsAt,hour,day,Zeit,Preis in ct
0,0.3056,0.0983,0.2073,2024-06-03 00:00:00+02:00,0,3,3. 0,30.56
1,0.2893,0.0846,0.2047,2024-06-03 01:00:00+02:00,1,3,3. 1,28.93
2,0.2882,0.0837,0.2045,2024-06-03 02:00:00+02:00,2,3,3. 2,28.82
3,0.2884,0.0838,0.2046,2024-06-03 03:00:00+02:00,3,3,3. 3,28.84
4,0.2864,0.0822,0.2042,2024-06-03 04:00:00+02:00,4,3,3. 4,28.64
5,0.3031,0.0962,0.2069,2024-06-03 05:00:00+02:00,5,3,3. 5,30.31
6,0.3472,0.1333,0.2139,2024-06-03 06:00:00+02:00,6,3,3. 6,34.72
7,0.4184,0.1931,0.2253,2024-06-03 07:00:00+02:00,7,3,3. 7,41.84
8,0.3844,0.1645,0.2199,2024-06-03 08:00:00+02:00,8,3,3. 8,38.44
9,0.3410,0.1281,0.2129,2024-06-03 09:00:00+02:00,9,3,3. 9,34.10


In [54]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.patches import Arc, Circle
from PIL import Image, ImageDraw, ImageFont
from datetime import datetime

# Function to create the circular gauge element
def create_circular_gauge(value, current_hour, max_value=1, size=(200, 200)):
    value = 20.75
    fig, ax = plt.subplots(figsize=(2, 2), subplot_kw={'aspect': 'equal'})
    ax.axis('off')

    # Draw the full circle in light gray
    ax.add_patch(Circle((0.5, 0.5), 0.4, color='#cccccc'))

    # Draw the arc in black
    angle = 360 * (value % 1 / max_value)
    
    print(f"{value=}\n{angle=}")
    arc = Arc((0.5, 0.5), 0.8, 0.8, angle=0, theta1=0, theta2=angle, color='black', linewidth=20)
    ax.add_patch(arc)

    # Add the text in black
    ax.text(0.5, 0.5, f'{value:.0f}ct', horizontalalignment='center', verticalalignment='center', fontsize=20, color='black')
    ax.text(0.5, 0.3, f'{current_hour}h', horizontalalignment='center', verticalalignment='center', fontsize=10, color='grey')

    # Save the circular gauge as an image
    fig.savefig('circular_gauge.png', bbox_inches='tight', pad_inches=0, transparent=True)
    plt.close(fig)

# Function to create the bar plot from a DataFrame
def create_bar_plot(df):
    current_hour = datetime.now().hour
    current_day = datetime.now().day
    
    hours = df['hour']
    days = df['day']
    prices_today = df['Preis in ct']

    fig, ax = plt.subplots(figsize=(8, 6))

    # Draw bars in grayscale
    ax.fill_between(hours, prices_today, step="mid", alpha=0.5, color='black', label='Heute')
    if current_hour in hours.values and current_day in days.values:
        current_price = df[df['hour'] == current_hour][df['day'] == current_day]['Preis in ct'].iloc[0]
        ax.bar(current_hour, current_price, color='black', edgecolor='white', linewidth=2)
    ax.set_ylabel('cent')
    ax.set_xlabel('Stunde')
    ax.set_title(f"Strompreisdaten vom {datetime.now().strftime('%d.%m.%Y')}")
    ax.legend(loc='upper left')

    ax.set_ylim(15, 45)

    # Save the plot as an image
    fig.savefig('bar_plot.png', bbox_inches='tight', format='png')
    plt.close(fig)

# Function to create the dashboard
def create_dashboard(df):
    # Get the last value for the circular gauge
    current_hour = datetime.now().hour
    current_day = datetime.now().day
    
    
    # Find the value for the current hour
    latest_value = df[df['hour'] == current_hour][df['day'] == current_day]['Preis in ct'].iloc[0]
    # Create the images
    create_circular_gauge(latest_value, current_hour)
    create_bar_plot(df)

    # Combine the images using Pillow
    dashboard_image = Image.new('L', (1200, 600), color='white')  # 'L' mode for grayscale
    draw = ImageDraw.Draw(dashboard_image)

    # Load the created images
    circular_gauge = Image.open('circular_gauge.png')#.convert('RGB')
    bar_plot = Image.open('bar_plot.png')#.convert('RGB')

    # Paste the images onto the dashboard
    dashboard_image.paste(circular_gauge, (50, 50), circular_gauge)  # The circular gauge
    dashboard_image.paste(bar_plot, (300, 50))  # The bar plot

    # Save the final dashboard image
    dashboard_image.save('dashboard_combined.png')

    
create_dashboard(df)


C:\Users\anton\AppData\Local\Temp\ipykernel_11660\4149570887.py:67: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  latest_value = df[df['hour'] == current_hour][df['day'] == current_day]['Preis in ct'].iloc[0]
C:\Users\anton\AppData\Local\Temp\ipykernel_11660\4149570887.py:46: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  current_price = df[df['hour'] == current_hour][df['day'] == current_day]['Preis in ct'].iloc[0]


value=20.75
angle=270.0


In [38]:
current_hour = datetime.now().hour
current_day = datetime.now().day


# Find the value for the current hour
latest_value = df[df['hour'] == current_hour][df['day'] == current_day]['Preis in ct'].iloc[0]
latest_value

C:\Users\anton\AppData\Local\Temp\ipykernel_11660\2540372349.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  latest_value = df[df['hour'] == current_hour][df['day'] == current_day]['Preis in ct'].iloc[0]


30.270000000000003

In [31]:
df[df['hour'] == current_hour][df['day'] == current_day]['Preis in ct'].iloc[0]

C:\Users\anton\AppData\Local\Temp\ipykernel_11660\1706762621.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['hour'] == current_hour][df['day'] == current_day]['Preis in ct'].iloc[0]


30.270000000000003